In [11]:
import torch
from torch.autograd.variable import Variable
import time
import math
import numpy as np

def VariableCast(value, grad = False):
    '''
    Casts an input to torch Variable object, with gradients either False or True. 
    input
    -----
    value - Type: scalar, Variable object, torch.Tensor, numpy ndarray
    grad  - Type: bool . If true then we require the gradient of that object
    output
    ------
    torch.autograd.variable.Variable object
    '''
    if value is None:
        return None
    elif isinstance(value, Variable):
        return value
    elif torch.is_tensor(value):
        return Variable(value, requires_grad = grad)
    elif isinstance(value, np.ndarray):
        tensor = torch.from_numpy(value).float()
        return Variable(tensor, requires_grad = grad)
    elif isinstance(value,list):
        return Variable(torch.FloatTensor(value), requires_grad=grad)
    else:
        return Variable(torch.FloatTensor([value]), requires_grad = grad)

In [12]:
class Integrator():

    def __init__(self, potential, min_step, max_step, max_traj, min_traj):
        self.potential = potential
        if min_step is None:
            self.min_step = np.random.uniform(0.01, 0.07)
        else:
            self.min_step = min_step
        if max_step is None:
            self.max_step = np.random.uniform(0.07, 0.18)
        else:
            self.max_step = max_step
        if max_traj is None:
            self.max_traj = np.random.uniform(18, 25)
        else:
            self.max_traj = max_traj
        if min_traj is None:
            self.min_traj = np.random.uniform(1, 18)
        else:
            self.min_traj = min_traj

    def generate_new_step_traj(self):
        ''' Generates a new step adn trajectory size  '''
        step_size = np.random.uniform(self.min_step, self.max_step)
        traj_size = int(np.random.uniform(self.min_traj, self.max_traj))
        return step_size, traj_size


    def leapfrog(self, p_init, x, grad_init):
        '''Performs the leapfrog steps of the HMC for the specified trajectory
        length, given by num_steps
        Parameters
        ----------
            values_init
            p_init
            grad_init     - Description: contains the initial gradients of the joint w.r.t parameters.

        Outputs
        -------
            x -    Description: proposed new x
            p      -    Description: proposed new auxillary momentum
        '''
        step_size, traj_size = self.generate_new_step_traj()
        values_init = x
        self.kinetic = Kinetic(p_init)
        # Start by updating the momentum a half-step and x by a full step
        p = p_init + 0.5 * step_size * grad_init
        x = values_init + step_size * self.kinetic.gauss_ke(p, grad=True)
        for i in range(traj_size - 1):
            # range equiv to [2:nsteps] as we have already performed the first step
            # update momentum
            p = p + step_size * self.potential.eval(x, grad=True)
            # update x
            x = x + step_size * self.kinetic.gauss_ke(p, grad=True)

        # Do a final update of the momentum for a half step
        p = p + 0.5 * step_size * self.potential.eval(x, grad=True)
        # print('Debug p, vlaues leapfrog', p, x)

        # return new proposal state
        return x, p

In [13]:
class Kinetic():
    ''' A basic class that implements kinetic energies and computes gradients
    Methods
    -------
    gauss_ke          : Returns KE gauss
    laplace_ke        : Returns KE laplace

    Attributes
    ----------
    p    - Type       : torch.Tensor, torch.autograd.Variable,nparray
        Size       : [1, ... , N]
        Description: Vector of current momentum

    M    - Type       : torch.Tensor, torch.autograd.Variable, nparray
        Size       : \mathbb{R}^{N \times N}
        Description: The mass matrix, defaults to identity.

    '''
    def __init__(self, p, M = None):

        if M is not None:
            if isinstance(M, Variable):
                self.M  = VariableCast(torch.inverse(M.data))
            else:
                self.M  = VariableCast(torch.inverse(M))
        else:
            self.M  = VariableCast(torch.eye(p.size()[0])) # inverse of identity is identity


    def gauss_ke(self,p, grad = False):
        '''' (p dot p) / 2 and Mass matrix M = \mathbb{I}_{dim,dim}'''
        self.p = VariableCast(p)
        P = Variable(self.p.data, requires_grad=True)
        K = 0.5 * P.t().mm(self.M).mm(P)

        if grad:
            return self.ke_gradients(P, K)
        else:
            return K
    def laplace_ke(self, p, grad = False):
        self.p = VariableCast(p)
        P = Variable(self.p.data, requires_grad=True)
        K = torch.sign(P).mm(self.M)
        if grad:
            return self.ke_gradients(P, K)
        else:
            return K
    def ke_gradients(self, P, K):
        return torch.autograd.grad([K], [P])[0]

In [14]:
def _grad_logp(self, logp, values):
    """
    Returns the gradient of the log pdf, with respect for
    each parameter

    Parameters
    ----------
    logjoint       - 
    values         - All parameters for which you need gradients.

    :return: torch.autograd.Variable
    """

    gradient_of_param = torch.autograd.grad(outputs=logp, inputs=values, retain_graph=True)[0]
    return gradient_of_param

In [15]:
class Metropolis():

    def __init__(self, potential, integrator, M):
        self.potential  = potential
        self.integrator = integrator
        self.M          = M
        self.count      = 0

    def sample_momentum(self,values):
        assert(isinstance(values, Variable))
        return Variable(torch.randn(values.data.size()))

    def hamiltonian(self, logjoint, p):
        """Computes the Hamiltonian  given the current postion and momentum
        H = U(x) + K(p)
        U is the potential energy and is = -log_posterior(x)
        Parameters
        ----------
        logjoint    - Type:torch.autograd.Variable
                    Size: \mathbb{R}^{1 \times D}
        p           - Type: torch.Tensor.Variable
                    Size: \mathbb{R}^{1 \times D}.
                    Description: Auxiliary momentum
        log_potential :Function from state to position to 'energy'= -log_posterior

        Returns
        -------
        hamitonian : float
        """
        T = self.kinetic.gauss_ke(p, grad=False)
        return -logjoint + T
    def acceptance(self, values_init, logjoint_init, grad_init):
        '''Returns the new accepted state

        Parameters
        ----------

        Output
        ------
        returns accepted or rejected proposal
        '''

        # generate initial momentum

        #### FLAG
        p_init = self.sample_momentum(values_init)
        # generate kinetic energy object.
        self.kinetic = Kinetic(p_init, self.M)
        # calc hamiltonian  on initial state
        orig = self.hamiltonian(logjoint_init, p_init)

        # generate proposals
        values, p = self.integrator.leapfrog(p_init, values_init, grad_init)

        # calculate new hamiltonian given current
        logjoint_prop, _ = self.potential.eval(values, grad=False)

        current = self.hamiltonian(logjoint_prop, p)
        alpha = torch.min(torch.exp(orig - current))
        # calculate acceptance probability
        if isinstance(alpha, Variable):
            p_accept = torch.min(torch.ones(1, 1), alpha.data)
        else:
            p_accept = torch.min(torch.ones(1, 1), alpha)
        # print(p_accept)
        if p_accept[0][0] > torch.Tensor(1, 1).uniform_()[0][0]:  # [0][0] dirty code to get integersr
            # Updates count globally for target acceptance rate
            # print('Debug : Accept')
            # print('Debug : Count ', self.count)
            self.count = self.count + 1
            return values, self.count
        else:
            # print('Debug : reject')
            return values_init, self.count

In [29]:
class HMCsampler():
    '''
    Notes:  the params from FOPPL graph will have to all be passed to - maybe
    Methods
    -------
    leapfrog_step - preforms the integrator step in HMC
    hamiltonian   - calculates the value of hamiltonian
    acceptance    - calculates the acceptance probability
    run_sampler

    Attributes
    ----------

    '''
    def __init__(self, program, burn_in= 100, n_samples= 20000, M= None,  min_step= None, max_step= None,\
                min_traj= None, max_traj= None):
        self.burn_in    = burn_in
        self.n_samples  = n_samples
        self.M          = M
        self.potential  = program()
        self.integrator = Integrator(self.potential, min_step, max_step, \
                                    min_traj, max_traj)
        # self.dim        = dim

        # TO DO : Implement a adaptive step size tuning from HMC
        # TO DO : Have a desired target acceptance ratio
        # TO DO : Implement a adaptive trajectory size from HMC


    def run_sampler(self):
        ''' Runs the hmc internally for a number of samples and updates
        our parameters of interest internally
        Parameters
        ----------
        n_samples
        burn_in

        Output
        ----------
        A tensor of the number of required samples
        Acceptance rate
        '''
        print(' The sampler is now running')
        # In the future dim = # of variables will not be needed as Yuan will provide
        # that value in the program and it shall return the required dim.
        logjoint_init, values_init, grad_init, dim = self.potential.generate()
        metropolis   = Metropolis(self.potential, self.integrator, self.M)
        temp,count   = metropolis.acceptance(values_init, logjoint_init, grad_init)
        samples      = Variable(torch.zeros(self.n_samples,dim))
        samples[0]   = temp.data.t()


        # Then run for loop from 2:n_samples
        for i in range(self.n_samples-1):
            logjoint_init, grad_init = self.potential.eval(temp, grad_loop= True)
            temp, count = metropolis.acceptance(temp, logjoint_init, grad_init)
            samples[i + 1, :] = temp.data.t()
            # try:
            #     samples[i+1,:] = temp.data.t()
            # except RuntimeError:
            #     print(i)
            #     break
            # update parameters and draw new momentum
            if i == np.floor(self.n_samples/4) or i == np.floor(self.n_samples/2) or i == np.floor(3*self.n_samples/4):
                print(' At interation {}'.format(i))

        # Basic summary statistics
        target_acceptance =  count / (self.n_samples)
        samples_reduced   = samples[self.burn_in:, :]
        mean = torch.mean(samples_reduced,dim=0, keepdim= True)
        print()
        print('****** EMPIRICAL MEAN/COV USING HMC ******')
        print('empirical mean : ', mean)
        print('Average acceptance rate is: ', target_acceptance)

        return samples,samples_reduced, mean

In [24]:
int_HMC = Integrator(1., None, None, None, None)

In [28]:
int_HMC.leapfrog

<bound method Integrator.leapfrog of <__main__.Integrator object at 0x7f79a6a64320>>